<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aula 8: Funções lambda, apply, applymap.

## Funções lambda

As funções lambda também são chamadas de funções anônimas. Uma função anônima é uma função definida sem um nome. Como sabemos, para definir uma função normal em Python, usamos a palavra-chave `def`. Mas, no caso de funções anônimas, usamos a palavra-chave `lambda` para defini-las.

As funções lambda são especialmente úteis na análise de dados porque, como veremos, há muitos casos em que as funções de transformação de dados tomarão **funções como argumentos**. Em geral, é mais claro passar uma função lambda em vez de escrever uma declaração de função completa ou atribuir uma expressão lambda a uma variável local.

### Sintaxe

`lambda arguments: expression`

Exemplos:

1) Definimos uma função lambda e a atribuímos à variável add. Na próxima linha, nós o executamos e imprimimos o resultado.

In [1]:
add = lambda a: a + a
print(add(20))

40


2) Definimos outra função lambda e armazenamos na variável add, executamos e imprimimos o resultado.

In [2]:
add = lambda a, b, c: a + b + c
print(add(10, 11, 12))

33


3) Reescrevemos a função `short_function` como uma função lambda.

In [3]:
def short_function(x):
    return x * 2

equiv_function = lambda x: x * 2

Em cada um dos exemplos de funções lambda:<br/>

- Quais são os argumentos?
- Qual é a expressão?

##  apply, applymap, map


O Pandas possui um conjunto de métodos que permitem operar de forma eficiente sobre os elementos de um objeto DataFrame ou Series.
Para aplicar a lógica desejada, podemos escolher definir funções nomeadas e usar expressões lambda que não podem ser reutilizadas.
    
1)  [pd.DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html): Opera em linhas ou colunas inteiras de um DataFrame

2)  [pd.DataFrame.applymap](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.applymap.html): Opera em cada um dos elementos de um DataFrame

3)  [pd.Series.apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html): Opera em cada um dos elementos de uma Series.
    
4)  [pd.Series.map](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html): Opera em cada um dos elementos de uma Series, muito semelhante a função .apply.
    
A diferença entre pd.Series.map e pd.Series.apply é que o segundo pode gerar um Dataframe a partir da série, enquanto o primeiro se recebesse uma Series como retorno da função, criaria uma série de Series.

O método pandas apply permite que você execute operações vetorizadas em objetos DataFrame linha por linha e coluna por coluna.

Veremos agora alguns exemplos de uso no conjunto de dados de propriedades de Melbourne, que usamos na prática de limpeza de dados.

In [4]:
import pandas as pd

# local
data_location = "../Data/melb_data.csv"
# colab
# data_location = ""

data = pd.read_csv(data_location)

data.head(3)


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Usando Apply para linhas de um objeto DataFrame

Vamos calcular o preço por metro quadrado de cada um dos registros no conjunto de dados.

Para isso, vamos dividir o valor do campo Price pelo valor do campo Landsize de cada registro.

Escrevemos essa conta como uma função lambda, que passaremos como o primeiro argumento a ser aplicado.

Na documentação, para o argumento do eixo (axis), vemos:

    0 or ‘index’: apply function to each column.

    1 or ‘columns’: apply function to each row.

Portanto, no segundo argumento, o valor do eixo é 1 porque queremos aplicar a função para as linhas.

Uma vez que temos que fazer uma divisão, devemos verificar se o denominador não é zero. Para isso, vamos escrever uma expressão condicional em uma linha com esta sintaxe:

`value_when_true if condition else value_when_false`

Neste caso:

`0 if x['Landsize'] == 0 else x['Price'] / x['Landsize']`


In [5]:
preco_m2 = data.apply(lambda x: 0 if x['Landsize'] == 0 else x['Price'] / x['Landsize'], axis = 1)
print(type(preco_m2))
print("Número de elementos em preco_m2: " + str(len(preco_m2)))
print("Quantidade de linhas em data: " + str(data.shape[0]))
print(preco_m2.head(3))

<class 'pandas.core.series.Series'>
Número de elementos em preco_m2: 13580
Quantidade de linhas em data: 13580
0     7326.732673
1     6634.615385
2    10932.835821
dtype: float64


Vemos que preco_m2 é uma instância da classe pd.Series e que tem o mesmo número de elementos que o número de linhas nos dados originais.

## Apply sobre colunas de um DataFrame

Agora vamos calcular a média de cada uma das colunas com valores numéricos.

Para determinar se uma coluna é do tipo numérico, vamos usar `np.issubdtype (..., np.number)` que retorna True se col_type for do tipo np.number, e False se não for. E vamos usar compreensão em listas para montar os nomes das colunas do tipo numérico:

`numeric_columns_mask = [np.issubdtype(data[col].dtypes, np.number) for col in data.columns]
numeric_columns_names = data.columns[numeric_columns_mask]`


Para isso definiremos uma função lambda que aplica `mean` em cada coluna cujo nome está na lista que construímos na etapa anterior.

Como queremos aplicar esta função em cada uma das colunas do DataFrame, o valor do eixo que usamos é 0.

In [6]:
import numpy as np

numeric_columns_mask = [np.issubdtype(data[col].dtypes, np.number) for col in data.columns]
numeric_columns_names = data.columns[numeric_columns_mask]
print(numeric_columns_names)

means = data.apply(lambda x: x.mean() if x.name in numeric_columns_names else np.NaN, axis=0)
print(means)
print(type(means))
print(len(means) == data.shape[1])

Index(['Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')
Suburb                    NaN
Address                   NaN
Rooms            2.937997e+00
Type                      NaN
Price            1.075684e+06
Method                    NaN
SellerG                   NaN
Date                      NaN
Distance         1.013778e+01
Postcode         3.105302e+03
Bedroom2         2.914728e+00
Bathroom         1.534242e+00
Car              1.610075e+00
Landsize         5.584161e+02
BuildingArea     1.519676e+02
YearBuilt        1.964684e+03
CouncilArea               NaN
Lattitude       -3.780920e+01
Longtitude       1.449952e+02
Regionname                NaN
Propertycount    7.454417e+03
dtype: float64
<class 'pandas.core.series.Series'>
True


Vemos que `mean` é uma instância de Series e que tem o mesmo número de elementos que o número de colunas nos dados

## Applymap sobre um objeto DataFrame

Agora vamos escrever todos os valores dos elementos do DataFrame em letras minúsculas.

Se o valor for numérico, não o alteramos.


In [7]:
data_lower = data.applymap(lambda x: x if np.isreal(x) else str(x).lower())
# comparo os tipos de dados antes e depois de aplicar a função .lower():
print(data_lower.dtypes == data.dtypes)
print(data_lower.head(3))

Suburb           True
Address          True
Rooms            True
Type             True
Price            True
Method           True
SellerG          True
Date             True
Distance         True
Postcode         True
Bedroom2         True
Bathroom         True
Car              True
Landsize         True
BuildingArea     True
YearBuilt        True
CouncilArea      True
Lattitude        True
Longtitude       True
Regionname       True
Propertycount    True
dtype: bool
       Suburb          Address  Rooms Type      Price Method SellerG  \
0  abbotsford     85 turner st      2    h  1480000.0      s  biggin   
1  abbotsford  25 bloomburg st      2    h  1035000.0      s  biggin   
2  abbotsford     5 charles st      3    h  1465000.0     sp  biggin   

        Date  Distance  Postcode  ...  Bathroom  Car  Landsize  BuildingArea  \
0  3/12/2016       2.5    3067.0  ...       1.0  1.0     202.0           NaN   
1  4/02/2016       2.5    3067.0  ...       1.0  0.0     156.0          79.0 

Vemos que os tipos de dados das colunas do DataFrame não foram modificados e que todas as cadeias de caracteres foram deixadas em minúsculas.

## Apply sobre um objeto Series


Construímos uma instância de Series com os valores da coluna Price.

Vamos aplicar um desconto de $10\%$ no preço, multiplicando cada um dos valores deste objeto Série por $0,9$.

In [8]:
price_serie = data.Price
print(type(price_serie))

price_discount = price_serie.apply(lambda x: x * 0.9)
price_discount

<class 'pandas.core.series.Series'>


0        1332000.0
1         931500.0
2        1318500.0
3         765000.0
4        1440000.0
           ...    
13575    1120500.0
13576     927900.0
13577    1053000.0
13578    2250000.0
13579    1156500.0
Name: Price, Length: 13580, dtype: float64

## Map sobre um objeto Series

Repetiremos o exemplo anterior com a função map.

In [9]:
price_serie = data.Price
print(type(price_serie))

price_discount = price_serie.map(lambda x: x * 0.9)
price_discount

<class 'pandas.core.series.Series'>


0        1332000.0
1         931500.0
2        1318500.0
3         765000.0
4        1440000.0
           ...    
13575    1120500.0
13576     927900.0
13577    1053000.0
13578    2250000.0
13579    1156500.0
Name: Price, Length: 13580, dtype: float64

## Usando a função apply com funções anônimas

Em todos os casos acima, usamos funções anônimas porque são executadas de forma mais eficiente do que aquelas definidas com `def`. No entanto, a sintaxe de apply, applymap e map é a mesma, independentemente da forma como a função é definida (`lambda` ou` def`).

A título de exemplo, vamos reescrever o exercício em que calculamos a média de cada uma das colunas de dados com valores numéricos, mas com uma função definida com "def" (não anônima).

In [10]:
def mean_numeric(column):    
    column_numeric = pd.to_numeric(column, errors = 'coerce') 
    # se não puder convertê-lo irá atribuir NaN a todos os elementos da coluna
    result = column_numeric.mean()
    return result

means = data.apply(mean_numeric, axis=0)
print(means)
print(type(means))
print(len(means) == data.shape[1])

Suburb                    NaN
Address                   NaN
Rooms            2.937997e+00
Type                      NaN
Price            1.075684e+06
Method                    NaN
SellerG                   NaN
Date                      NaN
Distance         1.013778e+01
Postcode         3.105302e+03
Bedroom2         2.914728e+00
Bathroom         1.534242e+00
Car              1.610075e+00
Landsize         5.584161e+02
BuildingArea     1.519676e+02
YearBuilt        1.964684e+03
CouncilArea               NaN
Lattitude       -3.780920e+01
Longtitude       1.449952e+02
Regionname                NaN
Propertycount    7.454417e+03
dtype: float64
<class 'pandas.core.series.Series'>
True


**Observação:**

É importante notar que os métodos que vimos nesta prática, applymap e map, são vetorizados e, portanto, executados de forma muito eficiente. É por isso que sempre os preferiremos em vez dos loops for ou while.

---

## Exercício

Vamos construir uma nova coluna (chamada postcode_suburb) nos dados do DataFrame, que é a concatenação do campo Postcode, seguido por "-", seguido pelo valor do campo Suburb.

Para isso, podemos a função apply ou uma função lambda.